In [1]:
import pandas as pd 
import os
import requests
import matplotlib.pyplot as plt 

In [2]:
lat = 49.083333
lon = -122.350
# lat: 49.083333    lon: -122.350
url = f'https://nominatim.openstreetmap.org/reverse?format=json&lat={lat}&lon={lon}&accept-language=en'
res = requests.get(url)



In [29]:
res.json()['address']['country']

'Canada'

In [3]:
income_US = pd.read_csv("./Energy_graph/data/metadata/kaggle_income.csv", encoding="latin-1")

income_US
def find_closest(df, lat, lon):
    df['distance'] = df.apply(lambda row: geodesic((lat, lon), (row['Lat'], row['Lon'])).miles, axis=1)
    closest_row = df.loc[df['distance'].idxmin()]
    return closest_row

In [21]:
gpd_PPP =  pd.read_csv("./Energy_graph/data/metadata/gdp-per-capita-worldbank.csv")
df = gpd_PPP[gpd_PPP["Year"] == 2020].drop(columns=["Year", "Code"])

df.reset_index(inplace=True, drop=True)
df.set_index("Entity", inplace=True)

In [37]:
df.loc[res.json()['address']['country']].values[0]

46064.254